In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from typing import Iterable
import libs.conv as conv

import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils import data

dev = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(dev)
print(device)

cpu


In [2]:
bsz, channels, depth, height, width = 4, 3, 224, 224, 224
input_size = (bsz, channels, depth, height, width)
padding = (2, 2, 2, 2, 2, 2)
kernel_size = (3, 3, 3)
stride = (1, 1, 1)
X = torch.rand(input_size)
print(X.shape)

torch.Size([4, 3, 224, 224, 224])


In [3]:
# class CustomConv3d(nn.Module):
#     def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation=1, groups=1, bias=False):
#         super().__init__()
#         self.__weight_dh = torch.nn.Parameter(torch.rand((out_channels, in_channels, kernel_size[0], kernel_size[1])))
#         self.__weight_dw = torch.nn.Parameter(torch.rand((out_channels, in_channels, kernel_size[0], kernel_size[2])))
#         self.__weight_hw = torch.nn.Parameter(torch.rand((out_channels, in_channels, kernel_size[1], kernel_size[2])))
#         self.weight = nn.ParameterList([self.__weight_dh, self.__weight_dw, self.__weight_hw])
#         self.bias = torch.nn.Parameter(torch.rand(out_channels)) if bias else None
#         self.__padding_dh = (padding[0], padding[1])
#         self.__padding_dw = (padding[0], padding[2])
#         self.__padding_hw = (padding[1], padding[2])
#         self.__stride_dh = (stride[0], stride[1])
#         self.__stride_dw = (stride[0], stride[2])
#         self.__stride_hw = (stride[1], stride[2])
#         self.__padding_mode = 'zeros'
#         self.__dilation = dilation
#         self.__groups = groups
    
#     def forward(self, X):
#         outdh, outdw, outhw = X.sum(dim=-1), X.sum(dim=-2), X.sum(dim=-3)
#         outdh = F.conv2d(outdh, self.__weight_dh, self.bias, self.__stride_dh, self.__padding_dh, self.__dilation, self.__groups)
#         outdw = F.conv2d(outdw, self.__weight_dw, self.bias, self.__stride_dw, self.__padding_dw, self.__dilation, self.__groups)
#         outhw = F.conv2d(outhw, self.__weight_hw, self.bias, self.__stride_hw, self.__padding_hw, self.__dilation, self.__groups)
#         out = outdh.unsqueeze(-1) + outdw.unsqueeze(-2) + outhw.unsqueeze(-3)
#         return out

In [4]:
def compare_times(layer1, layer2, n_trials):
    times1, times2 = [], []
    for _ in range(n_trials):
        start = time.time()
        out = layer1(X)
        times1.append(time.time() - start)

        start = time.time()
        out = layer2(X)
        times2.append(time.time() - start)
    return times1, times2

In [5]:
import time

custom_conv3d = conv.CustomConv3d(in_channels=3, out_channels=5, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
control_conv3d = nn.Conv3d(in_channels=3, out_channels=5, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
custom_times, control_times = [], []

n_trials = 1
custom_times, control_times = compare_times(custom_conv3d, control_conv3d, n_trials)
print(f'mean\ncustom: {np.mean(custom_times)}\ncontrol: {np.mean(control_times)}\nstd\ncustom: {np.std(custom_times)}\ncontrol: {np.std(control_times)}')

mean
custom: 0.6872379779815674
control: 1.5390276908874512
std
custom: 0.0
control: 0.0
